# Trying out Bayesian inference with PyMC3 on covid data

 _Disclaimer: this is in no way intended to be relied on!_
 _this was done purely for me to learn something_
 
It doesn't respect reactions of the countries, it doesn't respect the testing capabilities / numbers in the countries, it doesn't respect real biological models and past research in the field of virology and pandemics.

In [1]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
import pandas as pd
import theano
%matplotlib inline
import warnings
from scipy.stats import halfnorm
warnings.filterwarnings('ignore')

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Data based on a dump from a wiki page offering country specific infections.

Data is a snapshot form Kaggle taken from around mid April 2020 and wasn't updated since!

To make the data more representative, days before 2000 infections were reached were removed, since there might have been just single hotspots that were under control, also only those timeseries were looked at, that had in it's current state more than 20.000 infections counted.
Furthermore the data was restricted to series of at least 10 days.
These restrictions allow to look at a smaller set.

In [2]:
infections = []
countries = {}

MIN_DATES = 10

with open('untitled1.txt', 'r') as csv:
    intermediate = []
    counter = 0
    
    for line in csv:
        
        line = line.strip().split(',')
        country = line[2]+'-'+line[1]
        infection = int(float(line[4]))
        deaths = int(float(line[5]))
#        print(line)
                
        if infection < 2000:
            continue
        if not country in countries:
            countries[country] = 0
            counter = 0
            if len(intermediate) > MIN_DATES and intermediate[-1][2] > 10000:
                for i in intermediate:
                    infections.append(i)
                
            intermediate = []
        counter += 1
        intermediate.append([country, counter, infection, deaths])
    if len(intermediate) > MIN_DATES:
        for i in intermediate:
            infections.append(i)


        
full_df = None
full_df = pd.DataFrame(infections, columns=['country', 'day', 'infections', 'deaths'])
full_df = full_df.astype({'day': 'int32', 'infections': 'int32', 'deaths': 'int32'})

#filters = full_df.country.apply(lambda x: x in [
#        'China', 'Germany', 'Japan', 'South Korea', 'France', 'Netherlands'])
#full_df=full_df[filters]
countries = full_df.country.values
uniq_countries = full_df.country.unique()
n_countries = len(uniq_countries)

full_df['country_idx'] = [list(uniq_countries).index(x) for x in countries]
#print(full_df.country_idx)
#print(full_df)
print(list(enumerate(uniq_countries)))

[(0, 'Austria-'), (1, 'Belarus-'), (2, 'Belgium-'), (3, 'Brazil-'), (4, 'Canada-Ontario'), (5, 'Canada-Quebec'), (6, 'Chile-'), (7, 'China-Hubei'), (8, 'Ecuador-'), (9, 'France-'), (10, 'Germany-'), (11, 'India-'), (12, 'Iran-'), (13, 'Ireland-'), (14, 'Israel-'), (15, 'Italy-'), (16, 'Japan-'), (17, 'South Korea-'), (18, 'Mexico-'), (19, 'Netherlands-'), (20, 'Pakistan-'), (21, 'Peru-'), (22, 'Poland-'), (23, 'Portugal-'), (24, 'Qatar-'), (25, 'Romania-'), (26, 'Russia-'), (27, 'Saudi Arabia-'), (28, 'Singapore-'), (29, 'Spain-'), (30, 'Sweden-'), (31, 'Switzerland-'), (32, 'Turkey-'), (33, 'US-California'), (34, 'US-Colorado'), (35, 'US-Connecticut'), (36, 'US-Florida'), (37, 'US-Georgia'), (38, 'US-Illinois'), (39, 'US-Indiana'), (40, 'US-Louisiana'), (41, 'US-Maryland'), (42, 'US-Massachusetts'), (43, 'US-Michigan'), (44, 'US-New Jersey'), (45, 'US-New York'), (46, 'US-Ohio'), (47, 'US-Pennsylvania'), (48, 'US-Texas'), (49, 'US-Virginia'), (50, 'US-Washington'), (51, 'United Arab E

### here is the modeling part

the base idea is to fit a sigmoid like function to model the number of total infections. This assumption alone is probably already enough reason to not trust any output of this model. So _please don't trust_ the model.

Instead of using the regular sigmoid, I chose the _Gompertz Function_:

\begin{equation}
\large{
f(x) = a \cdot e^{b \cdot e^{c \cdot x} }
}
\end{equation}

The reason for using the Gompertz function is it's assymmetrie, allowing to adjust for the exponential increase ans slow down phases.
with $b, c < 0$ the value of $a$ determines the upper limit and therefore in our investigation the upper limit of infections.
$b$ and $c$ determine the speeed and acceleration.

To have some benefit from all the past countries, I tried to model $b$ and $c$ hierarchical, having a "mean value" across all time series, and the individual time series deviates from this according to a small normal distribution. The idea is, to have estimates for how things will develop even when very little hints are in the data.


In [ ]:
from theano import shared
predictors = full_df.day.values.copy()
predictors_shared = shared(predictors)
country_id = full_df.country_idx.values.copy()
country_idx = shared(country_id)


from theano import shared
predictors = full_df.day.values.copy()
predictors_shared = shared(predictors)



import scipy
with pm.Model() as model:
    
    a = pm.Uniform('a', lower=1000, upper=2000000, shape=n_countries)
    b_base = pm.Normal('b_base', mu=-4.5, sigma=0.5)
    b = pm.Normal('b', mu=b_base, sigma=0.5, shape=n_countries)
    c_base = pm.Normal('c_base', mu=-0.075, sigma=0.03)
    c = pm.Normal('c', mu=c_base, sigma=0.03, shape=n_countries)
    
    y = (a[country_idx] * pm.math.exp(b[country_idx] * pm.math.exp(c[country_idx] * (predictors_shared))))
    obs = pm.Normal('obs', mu=y, sigma=15000, observed=full_df.infections.values)

    trace = pm.sample(40000, cores=2)
    

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [c, c_base, b, b_base, a]
Sampling 2 chains, 0 divergences:   4%|▍         | 3538/81000 [02:59<1:04:25, 20.04draws/s]

### Now plotting the results of the fittings

The fittings did not work out very well, we will see why when we look at the traces.

We can see some pretty wide confidence intervals, so like the output suggested it didn't work out too well.
Interestingly this is especially then the case, when the counts haven't turned into the slow down phase where the infections are under control. This also makes sense, because the model has to guess which kind of behavior it will see when the infections get under control, without having any hints on it.
But here is the hierarchical model at least helping a bit, interpolating from overal behavior of the infections to the individual case.


In [ ]:
from pymc3 import forestplot

plt.figure(figsize=(20,20))
forestplot(trace, var_names=['a'])
forestplot(trace, var_names=['b'])
forestplot(trace, var_names=['c'])

pm.traceplot(trace)
print(list(enumerate(uniq_countries)))

### now predicting the future...

the traceplot above show what we already assumed, had some issues, especially the base values of c and b didn't fully converge to a single distribution, normally you would do a reparametrization and probably increase tuning steps to fix this.
But still let us try to now use the found model parameters to simulate how it's going to continue.


In [ ]:
#ppc = pm.sample_posterior_predictive(trace, samples=500, model=model)
x = np.tile(np.linspace(1, 100, 100).astype('int32'), n_countries)
print(len(x))
predictors_shared.set_value(x)

y = np.repeat(np.linspace(0,n_countries-1,n_countries).astype('int32'), 100)
print(len(y))
country_idx.set_value(y)

with model:
    post_pred = pm.sample_posterior_predictive(trace, samples=10000)

### looking at fittings and predictions
What we can actually see is that the model fitted the given points quite ok, but the predictions have quite a lot uncertainty. Especially in those cases, where there is little hint as to how much the region was able to slow down. 
So again don't rely on this model for anything.
This was done purely as an educational exercise.

In [ ]:

means = post_pred['obs'].mean(axis=0, keepdims=False).copy()
stds = post_pred['obs'].std(axis=0)
for i in range(n_countries):
    choice = y==i
    old_choice = full_df.country_idx==i

    plt.figure(figsize=(10,10))
    plt.errorbar(np.linspace(1,100,100),
                 means[choice],
                 stds[choice],
                 linestyle='None',
                 marker='.')
    plt.plot(np.linspace(1,len(full_df[old_choice]), len(full_df[old_choice])),
             full_df.infections[old_choice],
             marker='o')
    plt.title(uniq_countries[i])
    plt.show()



